In [1]:
!pip install --upgrade gtfs-realtime-bindings
!pip install mplleaflet

In [133]:
from google.transit import gtfs_realtime_pb2
import urllib.request
import mplleaflet as mp
import pandas as pd
import matplotlib.pyplot as plt
import time
import pickle

In [134]:
dir = '../data/static_data/'
route = pd.read_csv(dir + 'routes.txt',sep=",")
route

,route_short_name,route_long_name,route_type,route_id,agency_id
0,NaN,108DOWN,3,0,DIMTS
1,NaN,108STLDOWN,3,1,DIMTS
2,NaN,108STLUP,3,2,DIMTS
3,NaN,108UP,3,3,DIMTS
4,NaN,185DOWN,3,4,DIMTS
...,...,...,...,...,...
1540,NaN,624ACLDOWN_DTC,3,1540,DTC
1541,NaN,408CLDOWN_DTC,3,1541,DTC
1542,NaN,114UP_DTC,3,1543,DTC
1543,NaN,182ACLSTLU_DTC,3,1545,DTC


In [135]:
trip = pd.read_csv(dir + 'trips.txt',sep=',')
trip

,route_id,service_id,trip_id,shape_id
0,0,1,0_6_0,0.0
1,0,1,0_6_10,0.0
2,0,1,0_6_20,0.0
3,0,1,0_6_30,0.0
4,0,1,0_6_40,0.0
...,...,...,...,...
75752,1544,1,1544_15_25,NaN
75753,1544,1,1544_8_31,NaN
75754,1544,1,1544_17_7,NaN
75755,1544,1,1544_13_55,NaN


In [136]:
df=route.merge(trip,how='inner',on='route_id')[['route_long_name','route_id','shape_id']]
df

,route_long_name,route_id,shape_id
0,108DOWN,0,0.0
1,108DOWN,0,0.0
2,108DOWN,0,0.0
3,108DOWN,0,0.0
4,108DOWN,0,0.0
...,...,...,...
75752,114DOWN_DTC,1544,NaN
75753,114DOWN_DTC,1544,NaN
75754,114DOWN_DTC,1544,NaN
75755,114DOWN_DTC,1544,NaN


In [137]:
route['agency_id'].value_counts()

DIMTS    1210
DTC       335
Name: agency_id, dtype: int64

In [138]:
shapes = pd.read_csv(dir + 'shapes.txt', sep=",")
shapes

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,0,28.625198,77.110808,0
1,0,28.625220,77.110815,1
2,0,28.625280,77.110831,2
3,0,28.625483,77.110835,3
4,0,28.625705,77.110839,4
...,...,...,...,...
242239,533,28.618988,77.031115,708
242240,533,28.619000,77.031478,709
242241,533,28.619005,77.031547,710
242242,533,28.618953,77.031561,711


In [139]:
route_shape_map = df.merge(shapes,how='inner',on='shape_id')

In [140]:
route_shape_map.drop_duplicates(inplace=True)
route_shape_map

,route_long_name,route_id,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,108DOWN,0,0.0,28.625198,77.110808,0
1,108DOWN,0,0.0,28.625220,77.110815,1
2,108DOWN,0,0.0,28.625280,77.110831,2
3,108DOWN,0,0.0,28.625483,77.110835,3
4,108DOWN,0,0.0,28.625705,77.110839,4
...,...,...,...,...,...,...
16460992,751UP,533,533.0,28.618988,77.031115,708
16460993,751UP,533,533.0,28.619000,77.031478,709
16460994,751UP,533,533.0,28.619005,77.031547,710
16460995,751UP,533,533.0,28.618953,77.031561,711


# Graph creation

In [153]:
x = route_shape_map.groupby(['shape_pt_lat','shape_pt_lon']).size()
x

shape_pt_lat  shape_pt_lon
28.490695     77.304077        1
28.490785     77.303741       10
28.490846     77.304046        6
28.491244     77.303870       11
28.491317     77.304038        2
                              ..
28.858176     77.093328        1
              77.093332        1
              77.093337        1
              77.093342        1
              77.093347        1
Length: 45848, dtype: int64

In [154]:
graph = {}

for index,row in route_shape_map.iterrows():
    x = row.to_dict()
    gps = (x['shape_pt_lat'], x['shape_pt_lon'])
    if gps in graph:
        temp = x['route_id']
        if temp not in graph[gps]['route']:
            graph[gps]['route'].add(temp)
    else:
        graph[gps]={}
        graph[gps]['route']=set()
        graph[gps]['route'].add(x['route_id'])

In [156]:
%%time
for i, key in enumerate(graph):
    if i%1000 == 0:
        print(f'{i}/{len(graph)}')
    graph[key]['neg'] = []
    h = route_shape_map[(route_shape_map['shape_pt_lat']==key[0]) & (route_shape_map['shape_pt_lon']==key[1])].values.tolist()
    h2 = route_shape_map[route_shape_map['route_id'].isin(graph[key]['route'])]
    for row in h:
        temp = h2[(h2['route_id']==row[1]) & (h2['shape_pt_sequence']==row[-1]+1)]
        records = temp[['shape_pt_lat','shape_pt_lon']].to_records(index=False)
        result = list(records)
        graph[key]['neg'].extend(result)
for key in graph:
  graph[key]['neg'] = list(set([tuple(i) for i in graph[key]['neg']]))
graph

0/45848
1000/45848
2000/45848
3000/45848
4000/45848
5000/45848
6000/45848
7000/45848
8000/45848
9000/45848
10000/45848
11000/45848
12000/45848
13000/45848
14000/45848
15000/45848
16000/45848
17000/45848
18000/45848
19000/45848
20000/45848
21000/45848
22000/45848
23000/45848
24000/45848
25000/45848
26000/45848
27000/45848
28000/45848
29000/45848
30000/45848
31000/45848
32000/45848
33000/45848
34000/45848
35000/45848
36000/45848
37000/45848
38000/45848
39000/45848
40000/45848
41000/45848
42000/45848
43000/45848
44000/45848
45000/45848
CPU times: user 8min 24s, sys: 3.95 s, total: 8min 28s
Wall time: 8min 32s


211, 270, 416, 478},
  'neg': [(28.67114500000001, 77.184275)]},
 (28.67114500000001,
  77.184275): {'route': {3,
   10,
   23,
   208,
   211,
   270,
   416,
   478}, 'neg': [(28.6710978836767, 77.1842258173181),
   (28.67080300000001, 77.183918)]},
 (28.6710978836767, 77.1842258173181): {'route': {3, 10, 23, 270, 416, 478},
  'neg': [(28.67080300000001, 77.183918)]},
 (28.67080300000001,
  77.183918): {'route': {3,
   10,
   23,
   208,
   211,
   270,
   416,
   478}, 'neg': [(28.670665000000003, 77.183815)]},
 (28.670665000000003,
  77.183815): {'route': {3,
   10,
   23,
   208,
   211,
   270,
   416,
   478}, 'neg': [(28.670151, 77.183545)]},
 (28.670151, 77.183545): {'route': {3, 10, 23, 208, 211, 270, 416, 478},
  'neg': [(28.66998, 77.18345699999998)]},
 (28.66998,
  77.18345699999998): {'route': {3,
   10,
   23,
   208,
   211,
   214,
   270,
   416,
   468,
   472,
   478}, 'neg': [(28.669909000000004, 77.183421)]},
 (28.669909000000004,
  77.183421): {'route': {3,
   10

In [160]:
with open('graph.pkl','wb') as f:
  pickle.dump(graph,f)

In [161]:
with open('graph.pkl','rb') as f:
  graph = pickle.load(f)

In [162]:
for key in graph:
    print(graph[key])
    break

{'route': {0, 466, 532}, 'neg': [(28.62522, 77.110815)]}


##Trie


In [18]:
class TrieNode:
    Node_size = len(graph) 
    def __init__(self, loc=None, startRoutes=None, endRoutes=None):
        self.loc = loc
        self.neighbours = set()
        self.startRoutes = startRoutes
        self.endRoutes = endRoutes

class Trie:
    def __init__(self, graph):
        self.root = TrieNode()
    
    def charToIndex(self, ch):
        return ord(ch) - ord('a')

    def add(self, location, neghbours):
        pCrawl = self.root
        length = len(word)
        for n in neighbours:
            index = self.charToIndex(word[level])
            if pCrawl.neighbour[index]:
                pCrawl.children[index] = TrieNode()
            pCrawl = pCrawl.children[index]
        pCrawl.endOfWord = True
    
    def search(self, word):
        pCrawl = self.root
        length = len(word)
        for level in range(length):
            index = self.charToIndex(word[level])
            if not pCrawl.children[index]:
                return False
            pCrawl = pCrawl.children[index]
        return pCrawl.endOfWord

trie = Trie(graph)
for key in graph:
    trie.add(key, graph[key])

NameError: name 'word' is not defined

In [110]:
from sklearn.neighbors import KDTree
all_nodes = list(graph.keys())
kdtree = KDTree(all_nodes)

#Check


In [129]:
def put(feed, est_lat,est_lon, act_lat, act_lon):
  for entity in feed.entity:
    if entity.id=="DL1PD0569":
      ind = kdtree.query([[entity.vehicle.position.latitude,entity.vehicle.position.longitude]],return_distance=False)[0][0]
      lat1, lon1 = all_nodes[ind]
      est_lat.append(lat1)
      est_lon.append(lon1)
      act_lat.append(entity.vehicle.position.latitude)
      act_lon.append(entity.vehicle.position.longitude)
      break

In [104]:
feed = gtfs_realtime_pb2.FeedMessage()
response = urllib.request.urlopen('https://opendata.iiitd.edu.in/api/realtime/VehiclePositions.pb?key=5kOKha1VJqxWvkkIp1J71Yrb5mc3VsDS')
feed.ParseFromString(response.read())
print(feed.entity)

  route_id: "376"
  }
  position {
    latitude: 28.643327713012695
    longitude: 77.20238494873047
    speed: 0.0
  }
  timestamp: 1614158126
  vehicle {
    id: "DL1PD0639"
    label: "DL1PD0639"
  }
}
, id: "DL1PC5327"
vehicle {
  trip {
    trip_id: "363_14_52_7975_new"
    start_time: "14:52:45"
    start_date: "20210224"
    route_id: "363"
  }
  position {
    latitude: 28.529659271240234
    longitude: 77.28793334960938
    speed: 0.8333333134651184
  }
  timestamp: 1614158586
  vehicle {
    id: "DL1PC5327"
    label: "DL1PC5327"
  }
}
, id: "DL1PD3094"
vehicle {
  trip {
    trip_id: "448_14_50"
    start_time: "14:52:46"
    start_date: "20210224"
    route_id: "448"
  }
  position {
    latitude: 28.7346134185791
    longitude: 77.1976089477539
    speed: 0.0
  }
  timestamp: 1614158587
  vehicle {
    id: "DL1PD3094"
    label: "DL1PD3094"
  }
}
, id: "DL1PD4252"
vehicle {
  trip {
    trip_id: "555_14_52_6064_new"
    start_time: "14:52:46"
    start_date: "20210224"
   

In [131]:
feed = gtfs_realtime_pb2.FeedMessage()
est_lat=[]
est_lon=[]
act_lat = []
act_lon = []
for i in range(10):
  print(f'{i+1}/10')
  response = urllib.request.urlopen('https://opendata.iiitd.edu.in/api/realtime/VehiclePositions.pb?key=5kOKha1VJqxWvkkIp1J71Yrb5mc3VsDS')
  feed.ParseFromString(response.read())
  put(feed, est_lat, est_lon, act_lat, act_lon)
  print(act_lat[-1])
  time.sleep(10)

1/10
28.53357696533203
2/10
28.533126831054688
3/10
28.5329647064209
4/10
28.532882690429688
5/10
28.53281021118164
6/10
28.532390594482422
7/10
28.532390594482422
8/10
28.531471252441406
9/10
28.531471252441406
10/10
28.531021118164062


In [132]:
fig = plt.figure()
plt.plot(est_lon,est_lat,linewidth=3, color='b', marker='s')
plt.plot(act_lon,act_lat,linewidth=3, color='r', marker='s')
mp.display(fig=fig)